In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
db = sqlite3.connect("Monarchs2001.db")

In [3]:
df = pd.read_csv(r".\2001 Season Summary.csv")

        #convert pandas to SQL table
df.to_sql("monarchs2001", db, if_exists="replace")

os.getcwd



C:\Users\Arink\Anaconda3\lib\site-packages\pandas\core\generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


<function nt.getcwd()>

In [4]:
schema = pd.read_sql('SELECT * FROM sqlite_master ORDER by name', db)
schema

,type,name,tbl_name,rootpage,sql
0,index,ix_monarchs2001_index,monarchs2001,63,"CREATE INDEX ""ix_monarchs2001_index""ON ""monarc..."
1,index,ix_monarchs_2001_index,monarchs_2001,3,"CREATE INDEX ""ix_monarchs_2001_index""ON ""monar..."
2,table,monarchs2001,monarchs2001,62,"CREATE TABLE ""monarchs2001"" (\n""index"" INTEGER..."
3,table,monarchs_2001,monarchs_2001,2,"CREATE TABLE ""monarchs_2001"" (\n""index"" INTEGE..."


## Checking DataType

In [5]:
data_type = pd.read_sql('SELECT * FROM PRAGMA_TABLE_INFO("monarchs2001")', db)
data_type

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Tag No.,TEXT,0,None,0
2,2,Tagger,TEXT,0,None,0
3,3,"Tag Location, Date",TEXT,0,None,0
4,4,"Report Date, Location",TEXT,0,None,0
5,5,Reporter,TEXT,0,None,0
6,6,Miles,REAL,0,None,0


### Drop tagger and reporter identification

In [6]:
df.drop(["Tagger", "Reporter"], axis = 1, inplace = True)
df

,Tag No.,"Tag Location, Date","Report Date, Location",Miles
0,AAD929,"Salisbury, MA 09-12-01","02-11-02 El Rosario, MICH, MX",2344.0
1,AAD844,"Salisbury, MA 09-12-01","03-08-02 Sierra Chincua, MICH, MX",2343.0
2,AIO261,"Champlain, NY 09-24-01","03-13-02 El Rosario, MICH, MX",2329.0
3,AAU820,"Plattsburgh, NY 08-29-01","02-27-02 El Rosario, MICH, MX",2315.0
4,ABG581,"Saranac Lake, NY 09-16-01","03-13-02 El Rosario, MICH, MX",2303.0
...,...,...,...,...
1957,ADE225,NaN,"10-06-01 Lakeville, MN",NaN
1958,ABW153,NaN,"03-08-02 Sierra Chincua, MICH, MX",NaN
1959,AJA875,NaN,"02-26-02 El Rosario, MICH, MX",NaN
1960,ADR234,NaN,"02-26-02 El Rosario, MICH, MX",NaN


### Rename columns for clean-up

In [7]:
df.rename(columns={'Tag  Location, Date': 'tag_date', 'Report Date, Location': 'report_date'}, inplace=True)
df


,Tag No.,tag_date,report_date,Miles
0,AAD929,"Salisbury, MA 09-12-01","02-11-02 El Rosario, MICH, MX",2344.0
1,AAD844,"Salisbury, MA 09-12-01","03-08-02 Sierra Chincua, MICH, MX",2343.0
2,AIO261,"Champlain, NY 09-24-01","03-13-02 El Rosario, MICH, MX",2329.0
3,AAU820,"Plattsburgh, NY 08-29-01","02-27-02 El Rosario, MICH, MX",2315.0
4,ABG581,"Saranac Lake, NY 09-16-01","03-13-02 El Rosario, MICH, MX",2303.0
...,...,...,...,...
1957,ADE225,NaN,"10-06-01 Lakeville, MN",NaN
1958,ABW153,NaN,"03-08-02 Sierra Chincua, MICH, MX",NaN
1959,AJA875,NaN,"02-26-02 El Rosario, MICH, MX",NaN
1960,ADR234,NaN,"02-26-02 El Rosario, MICH, MX",NaN


### Split & rename combined columns in Tagged category

In [8]:
df2 = df['tag_date'].str.split(', ', expand=True)
df3 = df2[1].str.split(' ', expand=True)
df2.columns = ['tag_city', 'state_date']
df3.columns = ['Tagged State', 'Tagged Date']
df3


,Tagged State,Tagged Date
0,MA,09-12-01
1,MA,09-12-01
2,NY,09-24-01
3,NY,08-29-01
4,NY,09-16-01
...,...,...
1957,NaN,NaN
1958,NaN,NaN
1959,NaN,NaN
1960,NaN,NaN


### Split & Rename combined columns in Reported category

In [9]:
df4 = df['report_date'].str.split(', ', expand=True)
df5 = df4[0].str.extract('(\d\d-\d\d-\d\d)', expand=True)
#df6 = df4[0].str.replace('(\d\d-\d\d-\d\d)', '') - to seperate date from city. decided not to use city
df4.columns = ['city_date', 'Reported State', 'Reported Country']
df5.columns = ['Reported Date']

### Create new table with unneeded columns dropped

In [10]:
df7 = pd.concat([df, df2, df3, df4, df5], axis=1)
df7.drop(['state_date', 'tag_date', 'city_date', 'tag_city'], axis=1, inplace=True)
df8 = df7[['Tag No.','Tagged State', 'Tagged Date', 'Reported State', 'Reported Country', 'Reported Date', 'Miles']]
new_table = df8.copy()
new_table

,Tag No.,Tagged State,Tagged Date,Reported State,Reported Country,Reported Date,Miles
0,AAD929,MA,09-12-01,MICH,MX,02-11-02,2344.0
1,AAD844,MA,09-12-01,MICH,MX,03-08-02,2343.0
2,AIO261,NY,09-24-01,MICH,MX,03-13-02,2329.0
3,AAU820,NY,08-29-01,MICH,MX,02-27-02,2315.0
4,ABG581,NY,09-16-01,MICH,MX,03-13-02,2303.0
...,...,...,...,...,...,...,...
1957,ADE225,NaN,NaN,MN,None,10-06-01,NaN
1958,ABW153,NaN,NaN,MICH,MX,03-08-02,NaN
1959,AJA875,NaN,NaN,MICH,MX,02-26-02,NaN
1960,ADR234,NaN,NaN,MICH,MX,02-26-02,NaN


In [11]:
new_table["Tagged Date"]= pd.to_datetime(new_table["Tagged Date"]) 
new_table["Reported Date"]= pd.to_datetime(new_table["Reported Date"]) 
new_table.dtypes

Tag No.                     object
Tagged State                object
Tagged Date         datetime64[ns]
Reported State              object
Reported Country            object
Reported Date       datetime64[ns]
Miles                      float64
dtype: object

### Export cleaned data to csv.

In [12]:
new_table.to_csv("Monarchs2001.csv")